### 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB...

In [55]:
!apt install mongodb > log

In [56]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [57]:
!mongo

MongoDB shell version v3.6.8
connecting to: mongodb://127.0.0.1:27017
Implicit session: session { "id" : UUID("ab346757-9e9c-4bbc-b497-283bf9aad9f5") }
MongoDB server version: 3.6.8
Server has startup warnings: 
2023-03-01T18:25:16.341+0000 I CONTROL  [initandlisten] 
2023-03-01T18:25:16.341+0000 I CONTROL  [initandlisten] ** WARNING: Access control is not enabled for the database.
2023-03-01T18:25:16.341+0000 I CONTROL  [initandlisten] **          Read and write access to data and configuration is unrestricted.
2023-03-01T18:25:16.341+0000 I CONTROL  [initandlisten] 
> 2023-03-01T18:34:46.479+0000 I CONTROL  [main] shutting down with code:0


In [58]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [59]:
from pymongo import MongoClient

In [60]:
mongo_db = MongoClient()

In [61]:
mongo_db

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [62]:
db = mongo_db['vacancy_db']

In [63]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'vacancy_db')

In [65]:
list(mongo_db.list_database_names())

['admin', 'config', 'local']

In [66]:
db['vacancy']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'vacancy_db'), 'vacancy')

In [77]:
collection = db['vacancy']

In [88]:
collection.insert_one({'vacancy_name': 'Инженер-программист',
  'company_name': 'ФГБОУ ВО ТГМУ Минздрава России',
  'company_city': 'Владивосток',
  'salary_min': 20000,
  'salary_max': 30000,
  'salary_currency': '₽',
  'vacancy_link': 'https://hh.ru/search/vacancy/vakansii/inzhener-programmist-44190893.html',
  'site': 'https://www.superjob.ru/'})

In [79]:
from pprint import pprint

In [92]:
for i in collection.find(): # проверяем вывод добавленной вакансии
  pprint(i)

In [186]:
collection.delete_many({}) # удаляем все из коллекции

### ... и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

In [187]:
from bs4 import BeautifulSoup as bs
import requests
from lxml import html
import re

In [188]:
def parsing_hh():

    url = 'https://hh.ru/search/vacancy'
    headers = {
                'User-Agent': 
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
                }
    
    vacancy_search = []
    
    # ищем по ключевому слову Java
    
    params = { 
            'text': 'Java',              
            'items_on_page': '100', 
            'page': ''}

    response = requests.get(url, params = params, headers=headers)

    html = bs(response.text,'html.parser')

    # Зададим условие определения количество страниц с вакансиями в поисковой выдаче
    pages = html.find('div', {'data-qa': 'pager-block'}) # ищем блок с страницами

    if pages is None: # задаем условие проверки количества выдаваемых страниц с выбранной вакансией
        page_numbers = '1' # если блока страниц нет, значит вакансий меньше чем умещается на одной странице и принимаем
                           # количество страниц равным 1-ой
    else:
        page_numbers = int(pages.find_all('a', {'data-qa': 'pager-page'})[len(pages.find_all('a', {'data-qa': 'pager-page'}))-1].text)

    for page in range(0, int(page_numbers)): # задаем условие парсинга с первой страницы и далее
        params['page'] = page
        response = requests.get(url, params = params, headers=headers)
        html = bs(response.text,'html.parser')
        vacancy_list = html.find('div', {'data-qa': 'vacancy-serp__results'}).find_all('div', {'class': 'serp-item'}) 
        # задаем путь к нашим контейнерам, которые содержат контейнеры вакансий
     
        for i in vacancy_list: # и начинаем вытаскивать из каждого элемента листа вакансий, информацию по каждой вакансии
            
            vacancy_search_dict = {}

            vacancy_name = i.find('a', {'class': 'serp-item__title'}).getText()
            company_name = i.find('div', {'class': 'vacancy-serp-item__meta-info-company'}).getText().replace(u'\xa0', u' ')    
            company_city = i.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).getText()
            company_city = re.split(r', ', company_city)[0]
            
            # важно добавить условие нахождения самого тега <span>...данные по salary...<|span>, так как встречаются вакансии 
            # с комментарием "з/п не указана" с data-qa:"vacancy-salary-compensation-type-undefined" внутри карточки вакансии, 
            # но в списке поиска вакансий в блоке такой вакансии отсутствует <span> и если не проверять,то будет вылетать ошибка 
            
            if i.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation' }) is None:
                salary_min = None
                salary_max = None
                salary_currency = None
            else:
                vacancy_salary_1 = i.find_next('span', {'data-qa': 'vacancy-serp__vacancy-compensation' }) \
                .getText().replace('\u202f', '')
                vacancy_salary = re.split(r'\s|-', vacancy_salary_1)

                # разбиваем строку с зарплатой на список
                # так как ЗП может указываться в вариантах 300-400 руб. (тут будет len = 4), от 300 руб. (тут будет len = 3), 
                # до 400 руб. (тут будет тоже len = 3), то нужно прописать условия,
                # при которых будут выбираться минимальное и максимальное значение и валюта(так как длина списка будет меняться)
                
                if vacancy_salary[0] == 'до':
                    salary_min = None
                    salary_max = int(vacancy_salary[1])
                    salary_currency = vacancy_salary[2]
                elif vacancy_salary[0] == 'от':
                    salary_min = int(vacancy_salary[1])
                    salary_max = None
                    salary_currency = vacancy_salary[2]
                else:
                    salary_min = int(vacancy_salary[0])
                    salary_max = int(vacancy_salary[2])            
                    salary_currency = vacancy_salary[3]

            vacancy_link = i.find('a', {'class': 'serp-item__title'})['href']       

            vacancy_search_dict['vacancy_name'] = vacancy_name
            vacancy_search_dict['company_name'] = company_name
            vacancy_search_dict['company_city'] = company_city
            vacancy_search_dict['salary_min'] = salary_min
            vacancy_search_dict['salary_max'] = salary_max
            vacancy_search_dict['salary_currency'] = salary_currency
            vacancy_search_dict['vacancy_link'] = vacancy_link
            vacancy_search_dict['site'] = url.replace('/search/vacancy', '')
            vacancy_search.append(vacancy_search_dict)
            # тут прописываем правило, что если в коллекции уже встречается подобная ссылка на вакансию, то мы обновляем оставшиеся данные,
            # а если ссылки на вакансию в коллекции нет, то мы вносим в нее данную вакансию, таким образом записываются только новые вакансии
            # if collection.find_one({'vacancy_link': { "$in": [vacancy_search_dict['vacancy_link']]}}):
            if collection.find_one({'vacancy_link': vacancy_search_dict['vacancy_link']}):
                collection.update_one(
                    {'vacancy_link': vacancy_search_dict['vacancy_link']}, 
                    {'$set': {
                        'vacancy_name': vacancy_search_dict['vacancy_name'], 
                        'company_name': vacancy_search_dict['company_name'], 
                        'company_city': vacancy_search_dict['company_city'], 
                        'salary_min': vacancy_search_dict['salary_min'], 
                        'salary_max': vacancy_search_dict['salary_max'], 
                        'salary_currency': vacancy_search_dict['salary_currency'], 
                        'site': vacancy_search_dict['site']
                        }
                     }
                     )
            else:
                collection.insert_one(vacancy_search_dict)
    
            

    #return vacancy_search

In [189]:
parsing_hh()

In [206]:
#for i in collection.find():
    #pprint(i)   # закомментировал, чтобы не было большого списка

#### ниже вывод в виде DF, в подтверждении, что все работает:

In [205]:
import pandas as pd
df = pd.DataFrame(list(collection.find()))
df

,_id,vacancy_name,company_name,company_city,salary_min,salary_max,salary_currency,vacancy_link,site
0,63ffbac6aa0cef74782e2cdc,Middle/Senior Java разработчик,ООО CLICK,Ташкент,NaN,NaN,None,https://hh.ru/vacancy/77551177?from=vacancy_se...,https://hh.ru
1,63ffbac6aa0cef74782e2cdd,Senior Java Developer,ЗАО Водород,Минск,NaN,NaN,None,https://hh.ru/vacancy/75884361?from=vacancy_se...,https://hh.ru
2,63ffbac6aa0cef74782e2cde,Junior Java-разработчик,Аксиоматика,Волгоград,50000.0,NaN,руб.,https://hh.ru/vacancy/77480446?from=vacancy_se...,https://hh.ru
3,63ffbac6aa0cef74782e2cdf,Ведущий Java разработчик (удалённо),АО НТЦ ЕЭС Информационные Комплексы,Москва,350000.0,400000.0,руб.,https://hh.ru/vacancy/76643997?from=vacancy_se...,https://hh.ru
4,63ffbac6aa0cef74782e2ce0,Младший Java-разработчик ( плавающий график),ООО Темполайн Логистика,Москва,NaN,80000.0,руб.,https://hh.ru/vacancy/76966049?from=vacancy_se...,https://hh.ru
...,...,...,...,...,...,...,...,...,...
443,63ffbc9baa0cef74782e2e97,QA Automation инженер (Java),Bell Integrator,Москва,NaN,NaN,None,https://hh.ru/vacancy/77000881?from=vacancy_se...,https://hh.ru
444,63ffbc9baa0cef74782e2e98,Senior/Middle Java Developer (remote),ООО МКК А Деньги,Москва,NaN,300000.0,руб.,https://hh.ru/vacancy/76685797?from=vacancy_se...,https://hh.ru
445,63ffbc9baa0cef74782e2e99,Senior Backend разработчик/Java,ООО Гриндата,Санкт-Петербург,NaN,NaN,None,https://hh.ru/vacancy/75618834?from=vacancy_se...,https://hh.ru
446,63ffbc9baa0cef74782e2e9a,QA Auto (Java) engineer,IBS,Самара,NaN,NaN,None,https://hh.ru/vacancy/75776214?from=vacancy_se...,https://hh.ru


### 2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

In [191]:
def find_salary(salary, currency): # простая функция поиска зарплат в коллекции
    with_sal_min = collection.find({'salary_min': {'$gt': salary}, 'salary_currency': currency}) # ищутся все вакансии, где минимальный оклад больше указанного
    with_sal_max = collection.find({'salary_min': {'$gt': salary}, 'salary_currency': currency}) # ищутся все вакансии, где минимальный оклад больше указанного
    # также указываем валюту поиска, например 'руб.' или 'KZT'
    for i in with_sal_min or with_sal_max:  # собственно, если вакансия попала в выборку по минимуму или максимуму, то выдаем ее в запрос 
        pprint(i)

In [192]:
find_salary(250000, 'руб.')

{'_id': ObjectId('63ffbac6aa0cef74782e2cdf'),
 'company_city': 'Москва',
 'company_name': 'АО НТЦ ЕЭС Информационные Комплексы',
 'salary_currency': 'руб.',
 'salary_max': 400000,
 'salary_min': 350000,
 'site': 'https://hh.ru',
 'vacancy_link': 'https://hh.ru/vacancy/76643997?from=vacancy_search_list&query=Java',
 'vacancy_name': 'Ведущий Java разработчик (удалённо)'}
{'_id': ObjectId('63ffbac6aa0cef74782e2ce4'),
 'company_city': 'Москва',
 'company_name': 'BrainHub',
 'salary_currency': 'руб.',
 'salary_max': 450000,
 'salary_min': 350000,
 'site': 'https://hh.ru',
 'vacancy_link': 'https://hh.ru/vacancy/76610003?from=vacancy_search_list&query=Java',
 'vacancy_name': 'Senior Java developer'}
{'_id': ObjectId('63ffbac6aa0cef74782e2ce5'),
 'company_city': 'Москва',
 'company_name': 'ООО Робин',
 'salary_currency': 'руб.',
 'salary_max': None,
 'salary_min': 400000,
 'site': 'https://hh.ru',
 'vacancy_link': 'https://hh.ru/vacancy/77550595?from=vacancy_search_list&query=Java',
 'vacancy

### Проверка, что будут добавляться только новые вакансии/продукты в вашу базу

In [193]:
collection.find()[0] # берем первую вакансию в коллекции и запоминаем ссылку


{'_id': ObjectId('63ffbac6aa0cef74782e2cdb'), 'vacancy_name': 'Java-разработчик', 'company_name': 'ТОО KAP Technology', 'company_city': 'Астана', 'salary_min': None, 'salary_max': None, 'salary_currency': None, 'vacancy_link': 'https://hh.ru/vacancy/77457866?from=vacancy_search_list&query=Java', 'site': 'https://hh.ru'}

https://hh.ru/vacancy/77457866?from=vacancy_search_list&query=Java

теперь удаляем эту вакансию из коллекции и на ее место запишем новую с той же ссылкой, но с ошибками:

In [194]:
collection.delete_one({'vacancy_name': 'Java-разработчик', 
                       'company_name': 'ТОО KAP Technology', 
                       'company_city': 'Астана', 
                       'salary_min': None, 
                       'salary_max': None, 
                       'salary_currency': None, 
                       'vacancy_link': 'https://hh.ru/vacancy/77457866?from=vacancy_search_list&query=Java', 
                       'site': 'https://hh.ru'})

In [195]:
collection.find_one({'vacancy_link': 'https://hh.ru/vacancy/77457866?from=vacancy_search_list&query=Java'}) # проверяем, что вакансия удалена

In [196]:
collection.insert_one({'vacancy_name': 'Pizza courier', 
                       'company_name': 'dontknow', 
                       'company_city': 'dontknow', 
                       'salary_min': None, 
                       'salary_max': None, 
                       'salary_currency': None, 
                       'vacancy_link': 'https://hh.ru/vacancy/77457866?from=vacancy_search_list&query=Java', 
                       'site': 'https://hh.ru'})

In [197]:
collection.find_one({'vacancy_link': 'https://hh.ru/vacancy/77457866?from=vacancy_search_list&query=Java'}) # проверяем, что добавили вакансию с ошибкой

{'_id': ObjectId('63ffbc5aaa0cef74782e2e68'), 'vacancy_name': 'Pizza courier', 'company_name': 'dontknow', 'company_city': 'dontknow', 'salary_min': None, 'salary_max': None, 'salary_currency': None, 'vacancy_link': 'https://hh.ru/vacancy/77457866?from=vacancy_search_list&query=Java', 'site': 'https://hh.ru'}

Обратим внимание на новый '_id': ObjectId('63ffbc5aaa0cef74782e2e68')

In [199]:
parsing_hh() #включаем заново парсер

#### и проверяем вакансию в коллекции по данной ссылке

In [200]:
collection.find_one({'vacancy_link': 'https://hh.ru/vacancy/77457866?from=vacancy_search_list&query=Java'})

{'_id': ObjectId('63ffbc5aaa0cef74782e2e68'), 'vacancy_name': 'Java-разработчик', 'company_name': 'ТОО KAP Technology', 'company_city': 'Астана', 'salary_min': None, 'salary_max': None, 'salary_currency': None, 'vacancy_link': 'https://hh.ru/vacancy/77457866?from=vacancy_search_list&query=Java', 'site': 'https://hh.ru'}

#### обратим внимание, что парсер именно обновил данные и оставил прежний ID: <br>{'_id': ObjectId('63ffbc5aaa0cef74782e2e68)}
<br> При включении парсера снова, он будет добавлять только новые вакансии, а существующие вакансии обновлять, если спарсит по ним новые данные

In [201]:
for i in collection.find({'vacancy_link': 'https://hh.ru/vacancy/77457866?from=vacancy_search_list&query=Java'}):
    pprint(i)

{'_id': ObjectId('63ffbc5aaa0cef74782e2e68'),
 'company_city': 'Астана',
 'company_name': 'ТОО KAP Technology',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'https://hh.ru',
 'vacancy_link': 'https://hh.ru/vacancy/77457866?from=vacancy_search_list&query=Java',
 'vacancy_name': 'Java-разработчик'}
